## Table of Contents
* [Introduction: Business Problem](#Inroduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results](#result)
* [Conclusion](#conclusion)

# Introduction: Business Problem <a name="introduction"></a>

In this project we have to find optimal location for establishing a restaurant. The reports willl be targeted to the stakeholders are interested in opening a **French restaurant** in **Toronto, Ontario**.

As there are many French restaurants in Toronto, we will try to find out the areas that does not have any French restaurant. We would prefer areas that are located as close as to the centre.

# Data <a name = 'data'></a>

Based on the problem specified, we will require:
* Location data of Toronto
* Information of restaurants in Toronto

For **Location data** we will use `BeautifulSoup` library to extract **area information** from [here](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). For obtaining the coordinates of the areas will use [coordinate](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv) data.  
For obtaining the **information of restaurants in toronto** we will be using the **Foursquare API**.

#### Importing necessary libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)



import requests # library to handle requests
from bs4 import BeautifulSoup # library for webscrapping

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip3 install folium
import folium # map rendering library
print('libraries imported')

     |████████████████████████████████| 94 kB 5.3 MB/s  eta 0:00:01
libraries imported


#### Webscrapping areas

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source,'html.parser')

In [5]:
table_contents =[]
table=soup.find('table')
for row in table.findAll('td'):
    cell={}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
table_contents

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'},
 {'PostalCode': 'M4A',
  'Borough': 'North York',
  'Neighborhood': 'Victoria Village'},
 {'PostalCode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park, Harbourfront'},
 {'PostalCode': 'M6A',
  'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor, Lawrence Heights'},
 {'PostalCode': 'M7A',
  'Borough': "Queen's Park",
  'Neighborhood': 'Ontario Provincial Government'},
 {'PostalCode': 'M9A',
  'Borough': 'Etobicoke',
  'Neighborhood': 'Islington Avenue'},
 {'PostalCode': 'M1B',
  'Borough': 'Scarborough',
  'Neighborhood': 'Malvern, Rouge'},
 {'PostalCode': 'M3B',
  'Borough': 'North York',
  'Neighborhood': 'Don Mills North'},
 {'PostalCode': 'M4B',
  'Borough': 'East York',
  'Neighborhood': 'Parkview Hill, Woodbine Gardens'},
 {'PostalCode': 'M5B',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Garden District, Ryerson'},
 {'PostalCode': 'M6B', 'Borough': 'North York', 'N

Now, we will fit the extracted data to a dataframe using the `pandas` library and name the dataframe as **toronto**.

In [6]:
toronto = pd.DataFrame(table_contents)
toronto

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Let's find unique Boroughs to check if there is any inconsistency.

In [7]:
toronto['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East YorkEast Toronto', 'Central Toronto',
       'MississaugaCanada Post Gateway Processing Centre',
       'Downtown TorontoStn A PO Boxes25 The Esplanade',
       'EtobicokeNorthwest',
       'East TorontoBusiness reply mail Processing Centre969 Eastern'],
      dtype=object)

As we can see there are a few inconsistencies, we will try to correct it.

In [8]:
toronto['Borough'] = toronto['Borough'].replace({
    'MississaugaCanada Post Gateway Processing Centre': 'Mississauga',
    'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto',
    'EtobicokeNorthwest': 'Etobicoke Northwest',
    'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto',
    'East YorkEast Toronto': 'East York/East Toronto'
})
toronto['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East York/East Toronto', 'Central Toronto', 'Mississauga',
       'Etobicoke Northwest'], dtype=object)

In [9]:
toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Now, let's extract the coordinates of areas from the csv file given below and name it as **location_df**.

In [10]:
location_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
location_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now, that we have the coordinates we will try to merge **toronto** and **location_df** dataframes, based on their postal codes.  In order to do so, first we have to match the `PostalCode` column of both the dataframes.

In [11]:
location_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
toronto = pd.merge(toronto, location_df, on='PostalCode')
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Now, let's find the centre coodinates of **Toronto, Ontario** using `geopy.geocoders.Nominatim` library.

In [12]:
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
center_coordinate = [latitude, longitude]
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


Ok, now let us **visualize** the data that we have so far i.e. areas and neighborhoods of the toronto data using the `folium` library.

In [13]:
import folium
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)

toronto_map

### Foursquare API

Now that we have the coodinates of the location of areas in Toronto, we can get the informations about restaurants each area using the **Foursqaure API**.  
Using this api we will exatract the following-
* Areas around the coordinates
* Venues in the areas
* Coordinates of venues
* Category of the venues

In [14]:
# The code was removed by Watson Studio for sharing.

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                 latitudes=toronto['Latitude'], longitudes=toronto['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [17]:
print(toronto_venues.shape)
toronto_venues.head()

(2142, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


# Methodology <a name='methodology'></a>

In this project we will direct our efforts on detecting areas in **Toronto, Ontario** that does not have any French restaurants and also limiting our search to within **6km radius** from the **city centre**.  
First we will filter the **venue category** with the word **Restaurant** and according to that we will analyze our data.

In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


We will now filter the **toronto venues** based on the **venue category** in reference to the word **Restaurant** and name it as `toronto_restaurants`.

In [19]:
toronto_restaurants= toronto_venues[toronto_venues['Venue Category'].str.contains('Restaurant')]
toronto_restaurants.head()

,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
12,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
29,"Regent Park, Harbourfront",43.654260,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


This gives us the areas that have restaurants.

Again, we will further filter the `toronto_restaurants` data based on the **venue category** with the word **French** and name it as `french_restaurant`.  
This will indeed give us area details that have **French Restaurants**.

In [20]:
french_restaurant=toronto_restaurants[toronto_restaurants['Venue Category'].str.contains('French')]
french_restaurant

,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
29,"Regent Park, Harbourfront",43.654260,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
314,St. James Town,43.651494,-79.375418,Biff's Bistro,43.647085,-79.376342,French Restaurant
386,Berczy Park,43.644771,-79.373306,Biff's Bistro,43.647085,-79.376342,French Restaurant
501,Central Bay Street,43.657952,-79.387383,Midi Bistro,43.655871,-79.392091,French Restaurant
916,"Little Portugal, Trinity",43.647927,-79.419750,La Banane,43.649040,-79.420599,French Restaurant
1070,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,Biff's Bistro,43.647085,-79.376342,French Restaurant
1206,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Biff's Bistro,43.647085,-79.376342,French Restaurant
1569,"University of Toronto, Harbord",43.662696,-79.400049,Cafe Cancan,43.662735,-79.403447,French Restaurant
1603,"Runnymede, Swansea",43.651571,-79.484450,Amber European Restaurant,43.649946,-79.482009,French Restaurant


# Analysis <a name='analysis'></a>

In [21]:
print('Total number of restaurants', len(toronto_restaurants))
print('Total number of French restaurants', len(french_restaurant))
print('Percentage of French restaurants: {:.2f}%'.format(len(french_restaurant)/len(toronto_restaurants) * 100))

Total number of restaurants 500
Total number of French restaurants 11
Percentage of French restaurants: 2.20%


Let's visualize the `toronto_restaurant` data

In [22]:
toronto_restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, venCat, area in zip(toronto_restaurants['Venue Latitude'], toronto_restaurants['Venue Longitude'], toronto_restaurants['Venue Category'], toronto_restaurants['Area']):
    label = '{},\n{}'.format(venCat, area)
    label = folium.Popup(label, parse_html=True)
    color = 'blue' if venCat=='French Restaurant' else 'red'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color = color,
        fill=True,
        fill_color = color,
        fill_opacity=0.5,
        parse_html=False).add_to(toronto_restaurant_map)

for lat, lng, venCat, area in zip(french_restaurant['Venue Latitude'], french_restaurant['Venue Longitude'], french_restaurant['Venue Category'], french_restaurant['Area']):
    folium.Marker([lat, lng], popup='{}, {}'.format(venCat, area)).add_to(toronto_restaurant_map)
toronto_restaurant_map

Here, the locations that are marked with popups, are the actual locations of **French Restaurants**.  And the locations that are marked with with color **red** are locations of **non-french restaurants**.

From here two cases may arise-  
**CASE I** : The stakeholders doesn't have problem in having restaurants other than french to exist in the prefered area.   
**CASE II** : The stakeholders do not want any type of restaurant in the prefered area.

Now, since we do not require the areas that have french restaurants, therefore we will eliminate that areas from the prefered area list.

In [23]:
french_restaurant_area = list(french_restaurant['Area'])
french_restaurant_area

['Victoria Village',
 'Regent Park, Harbourfront',
 'St. James Town',
 'Berczy Park',
 'Central Bay Street',
 'Little Portugal, Trinity',
 'Toronto Dominion Centre, Design Exchange',
 'Commerce Court, Victoria Hotel',
 'University of Toronto, Harbord',
 'Runnymede, Swansea',
 'Enclave of M5E']

##### CASE I
The stakeholders doesn't have problem in having restaurants other than french to exist in the prefered area.

In [24]:
non_french_restaurant_area = toronto_venues[~toronto_venues.Area.isin(french_restaurant_area)]
non_french_restaurant_area

,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
55,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Roots,43.718214,-79.463893,Boutique
56,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Kitchen Stuff Plus (Clearance Outlet),43.719096,-79.462675,Furniture / Home Store
...,...,...,...,...,...,...,...
2137,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Subway,43.631659,-79.519001,Sandwich Place
2138,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Jim & Maria's No Frills,43.631152,-79.518617,Grocery Store
2139,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Islington Florist & Nursery,43.630156,-79.518718,Flower Shop
2140,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon


In [25]:
recommended_areas1 = non_french_restaurant_area[['Area', 'Area Latitude', 'Area Longitude']]
recommended_areas1 = recommended_areas1.groupby('Area', as_index= False).first()
recommended_areas1

,Area,Area Latitude,Area Longitude
0,Agincourt,43.794200,-79.262029
1,"Alderwood, Long Branch",43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,Bayview Village,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
5,"Birch Cliff, Cliffside West",43.692657,-79.264848
6,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
7,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
8,Caledonia-Fairbanks,43.689026,-79.453512
9,Cedarbrae,43.773136,-79.239476


Ok, we got the area name and locations that have no french restaurants.  
Now, let's try to calculate the distance of each area from the city centre.

In [26]:
import math
def calc_distance(lat1, lat2, lng1, lng2):
    r = 6378.1  # radius of earth
    x1 = math.radians(lat1)
    x2 = math.radians(lat2)
    y1 = math.radians(lng1)
    y2 = math.radians(lng2)
    
    dis_lat = x2 - x1
    dis_lng = y2 - y1
    
    a = math.sin(dis_lat / 2)**2 + math.cos(x1) * math.cos(x2) * math.sin(dis_lng / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = r*c
    return distance

In [27]:
center_coordinate

[43.6534817, -79.3839347]

In [28]:
distances =[]
for lat, lng in zip(recommended_areas1['Area Latitude'], recommended_areas1['Area Longitude']):
    distance = calc_distance(center_coordinate[0], lat, center_coordinate[1], lng)
    distances.append(distance)

In [29]:
recommended_areas1['Distance from Ontario Center'] = distances
recommended_areas1.head()

,Area,Area Latitude,Area Longitude,Distance from Ontario Center
0,Agincourt,43.794200,-79.262029,18.481292
1,"Alderwood, Long Branch",43.602414,-79.543484,14.056752
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,12.167837
3,Bayview Village,43.786947,-79.385975,14.858143
4,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,9.339357


According to our analysis, these are the recommended areas.  
But, the areas prefered by the stakeholders are given below, which is **within the 6 km radius** from the center. 

In [30]:
prefered_areas1 = recommended_areas1[recommended_areas1['Distance from Ontario Center'] <= 6]
prefered_areas1 = prefered_areas1.sort_values(by='Distance from Ontario Center')
prefered_areas1

,Area,Area Latitude,Area Longitude,Distance from Ontario Center
62,"Richmond, Adelaide, King",43.650571,-79.384568,0.327978
32,"Garden District, Ryerson",43.657162,-79.378937,0.574313
30,"First Canadian Place, Underground city",43.648429,-79.382280,0.578010
58,Ontario Provincial Government,43.662301,-79.389494,1.079071
44,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,1.298273
11,Church and Wellesley,43.665860,-79.383160,1.379339
36,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,1.420881
69,"St. James Town, Cabbagetown",43.667967,-79.367675,2.077179
73,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,2.765412
7,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,2.858841


In [31]:
prefered_area1_map = folium.Map(center_coordinate, zoom_start=12)
folium.Marker(center_coordinate, popup='Toronto, Ontario').add_to(prefered_area1_map)
for area, lat, lng, dist in zip(prefered_areas1['Area'], prefered_areas1['Area Latitude'],
                               prefered_areas1['Area Longitude'], prefered_areas1['Distance from Ontario Center']):
    label = 'area: {}, \ndistance from center: {} km'.format(area, round(dist, 2))
    folium.CircleMarker(
    [lat, lng],
    radius=10,
    popup=label,
    color = 'green',
    fill=True,
    fill_color = 'green',
    fill_opacity=0.7,
    parse_html=False).add_to(prefered_area1_map)
prefered_area1_map

Here, the areas coloured in **green** represents the **prefered areas**, and the popup is the center i.e, **Toronto, Ontario**.

Now, let's find the **top 5** exact street addresses of the **prefered areas**.  
For that we will be using reverse geocoding.

In [32]:
def get_address(lat, lng):
    from geopy.geocoders import Nominatim
    locator = Nominatim(user_agent='myGeocoder')
    coordinates = str(lat)+', '+str(lng)
    loction = locator.reverse(coordinates)
    return loction.address

In [33]:
areas = []
for lat, lng in zip(prefered_areas1['Area Latitude'], prefered_areas1['Area Longitude']):
    addresses = get_address(lat, lng)
    areas.append(addresses)
print('Address Obtained!')

Address Obtained!


In [34]:
top5areas = areas[:5]

In [35]:
pd.set_option("display.max_colwidth", None)
pd.DataFrame({
    'Top 5 recommended areas':top5areas
})

,Top 5 recommended areas
0,"167, York Street, Financial District, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5H, Canada"
1,"Ryerson University, Frank Natale Lane, Garden District, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5B 1E5, Canada"
2,"Prairie Girl Bakery, PATH, Financial District, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5H, Canada"
3,"Sheela Basrur, Grosvenor Street, Discovery District, University—Rosedale, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5S 1Z6, Canada"
4,"25, Kensington Avenue, Kensington Market, University—Rosedale, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5T 2K2, Canada"


These are the **top 5 addresses** where **no french restaurant** exist.

##### CASE II
The stakeholders do not want any type of restaurant in the prefered area.

To do so, we first have to get the list of areas that have restaurants, and then we will eliminate them from the prefered list.

In [36]:
restaurant_area = list(toronto_restaurants['Area'])
restaurant_area

['Parkwoods',
 'Victoria Village',
 'Victoria Village',
 'Regent Park, Harbourfront',
 'Regent Park, Harbourfront',
 'Regent Park, Harbourfront',
 'Regent Park, Harbourfront',
 'Regent Park, Harbourfront',
 'Lawrence Manor, Lawrence Heights',
 'Ontario Provincial Government',
 'Ontario Provincial Government',
 'Ontario Provincial Government',
 'Ontario Provincial Government',
 'Ontario Provincial Government',
 'Ontario Provincial Government',
 'Ontario Provincial Government',
 'Malvern, Rouge',
 'Don Mills North',
 'Don Mills North',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerson',
 'Garden District, Ryerso

In [37]:
non_restaurant_area = toronto_venues[~toronto_venues.Area.isin(restaurant_area)]
non_restaurant_area.head(10)

,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
104,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Jawny Bakers,43.705783,-79.312913,Gastropub
105,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,East York Gymnastics,43.710654,-79.309279,Gym / Fitness Center
106,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Shoppers Drug Mart,43.705933,-79.312825,Pharmacy
107,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,TD Canada Trust,43.705740,-79.312270,Bank
108,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Pizza Pizza,43.705159,-79.313130,Pizza Place
109,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Rise & Dine Eatery,43.705769,-79.311638,Breakfast Spot
110,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Nostalgia,43.706833,-79.311783,Café
111,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,East York Animal Clinic,43.705921,-79.312196,Pet Store
112,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,St. Clair Ave E & O'Connor Dr,43.705233,-79.313274,Intersection
113,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Venice Pizza,43.705921,-79.313957,Pizza Place


In [38]:
recommended_areas2 = non_restaurant_area[['Area', 'Area Latitude', 'Area Longitude']]
recommended_areas2 = recommended_areas2.groupby('Area', as_index= False).first()
recommended_areas2.head()

,Area,Area Latitude,Area Longitude
0,"Alderwood, Long Branch",43.602414,-79.543484
1,"Birch Cliff, Cliffside West",43.692657,-79.264848
2,"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",43.628947,-79.394420
3,Caledonia-Fairbanks,43.689026,-79.453512
4,"Clairville, Humberwood, Woodbine Downs, West Humber, Kipling Heights, Rexdale, Elms, Tandridge, Old Rexdale",43.706748,-79.594054


In [39]:
distances =[]
for lat, lng in zip(recommended_areas2['Area Latitude'], recommended_areas2['Area Longitude']):
    distance = calc_distance(center_coordinate[0], lat, center_coordinate[1], lng)
    distances.append(distance)

recommended_areas2['Distance from Ontario Center'] = distances
recommended_areas2.head()

,Area,Area Latitude,Area Longitude,Distance from Ontario Center
0,"Alderwood, Long Branch",43.602414,-79.543484,14.056752
1,"Birch Cliff, Cliffside West",43.692657,-79.264848,10.533510
2,"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",43.628947,-79.394420,2.858841
3,Caledonia-Fairbanks,43.689026,-79.453512,6.858629
4,"Clairville, Humberwood, Woodbine Downs, West Humber, Kipling Heights, Rexdale, Elms, Tandridge, Old Rexdale",43.706748,-79.594054,17.925154


In [40]:
prefered_areas2 = recommended_areas2[recommended_areas2['Distance from Ontario Center'] <= 6]
prefered_areas2 = prefered_areas2.sort_values(by='Distance from Ontario Center')
prefered_areas2

,Area,Area Latitude,Area Longitude,Distance from Ontario Center
2,"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",43.628947,-79.394420,2.858841
21,Rosedale,43.679563,-79.377529,2.948756
27,The Danforth East,43.685347,-79.338106,5.118568
14,Humewood-Cedarvale,43.693781,-79.428191,5.729094


In [41]:
prefered_area2_map = folium.Map(center_coordinate, zoom_start=12)
folium.Marker(center_coordinate, popup='Toronto, Ontario').add_to(prefered_area2_map)
for area, lat, lng, dist in zip(prefered_areas2['Area'], prefered_areas2['Area Latitude'],
                               prefered_areas2['Area Longitude'], prefered_areas2['Distance from Ontario Center']):
    label = 'area: {}, \ndistance from center: {} km'.format(area, round(dist, 2))
    folium.CircleMarker(
    [lat, lng],
    radius=10,
    popup=label,
    color = 'green',
    fill=True,
    fill_color = 'green',
    fill_opacity=0.7,
    parse_html=False).add_to(prefered_area2_map)
prefered_area2_map

Here, the areas coloured in **green** represents the **areas** that are closest to the center and have **no restaurants**.  

In [42]:
areas = []
for lat, lng in zip(prefered_areas2['Area Latitude'], prefered_areas2['Area Longitude']):
    addresses = get_address(lat, lng)
    areas.append(addresses)

pd.set_option("display.max_colwidth", None)
pd.DataFrame({
    'Top 5 recommended areas':areas
})

,Top 5 recommended areas
0,"Billy Bishop Toronto City Airport, Lakeshore Avenue, Spadina—Fort York, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5V 1A1, Canada"
1,"20, Lamport Avenue, Rosedale, University—Rosedale, Old Toronto, Toronto, Golden Horseshoe, Ontario, M4W 1R1, Canada"
2,"43, Marlow Avenue, Toronto—Danforth, East York, Toronto, Golden Horseshoe, Ontario, M4J 3P8, Canada"
3,"28, Glen Cedar Road, Cedarvale, Toronto—St. Paul's, York, Toronto, Golden Horseshoe, Ontario, M6C 1W4, Canada"


These are the **top 5** street addresses of areas that do not have any restaurants.

# Results <a name='result'></a>

Thoroughout our analysis, we have noticed that there are many number of restaurants concentrated around the center. This analysis led us to two different cases, which were :    
* The stakeholders doesn't have problem in having restaurants other than french to exist in the prefered area.  
* The stakeholders do not want any type of restaurant in the prefered area.  

By the end of the analysis of each cases, we narrowed down our searches of prefered areas to a radius of 6 km from the center. And from there we, obtained top 5 areas closest to the city center, best suitable for opening a **French restaurant**. 

The prefered areas of **CASE I** are the areas close to the center, but, have restaurants (non-french). On the other hand, the prefered areas of **CASE II** are the areas close to the center and also doesn't have any restaurants.

# Conclusion <a name = 'conclusion'></a> 

The purpose of this project was to find best areas in Toronto, suitable for opening a French restaurant. By calculating restaurant density from Foursquare data, we have first eliminated the areas that had French restaurants. The rest of the areas were used for futher analysis, to get the desired areas.  
We have already identified the top 5 suitable areas for the establishment of French restaurant. 

Now, the final decision of choosing the most favourable area will be done by the stakeholders based on the budget, availability of essenstial things in that area, tourists visit etc.